In [1]:
import numpy as np
import matplotlib.pyplot as plt
import glob
from specim_test.specim.specfuncs import spec2d
from astropy.io import fits as pf

In [2]:
file_loc = '../galaxy_spectra/DEIMOS_data/2M1310_data/*'

In [3]:
file_list = []
out_file_det2 = []
out_file_det6 = []

for i,p in enumerate(glob.glob(file_loc)):
    if 'spec2d' in p and '2M1310' in p:
        file_list.append(p)
        f = p.split('_')
        frame = f[-4]+ '_' + f[-3][:4]
        out_file_det2.append('2M1310_DEIMOS_spectra/2M1310_deimos_det2_'+ frame + '_clean.fits')
        out_file_det6.append('2M1310_DEIMOS_spectra/2M1310_deimos_det6_'+ frame + '_clean.fits')

In [4]:
file_list = sorted(file_list)
file_list

['../galaxy_spectra/DEIMOS_data/2M1310_data/spec2d_d0613_0074-2M1310m1_DEIMOS_20210613T060655.930.fits',
 '../galaxy_spectra/DEIMOS_data/2M1310_data/spec2d_d0613_0075-2M1310m1_DEIMOS_20210613T063837.766.fits',
 '../galaxy_spectra/DEIMOS_data/2M1310_data/spec2d_d0614_0075-2M1310m1_DEIMOS_2021Jun14T065013.891.fits']

In [42]:
out_file_det2 = sorted(out_file_det2)
out_file_det6 = sorted(out_file_det6)
out_file_det2, out_file_det6

(['2M1310_DEIMOS_spectra/2M1310_deimos_det2_d0613_0074_clean.fits',
  '2M1310_DEIMOS_spectra/2M1310_deimos_det2_d0613_0075_clean.fits',
  '2M1310_DEIMOS_spectra/2M1310_deimos_det2_d0614_0075_clean.fits'],
 ['2M1310_DEIMOS_spectra/2M1310_deimos_det6_d0613_0074_clean.fits',
  '2M1310_DEIMOS_spectra/2M1310_deimos_det6_d0613_0075_clean.fits',
  '2M1310_DEIMOS_spectra/2M1310_deimos_det6_d0614_0075_clean.fits'])

### finding out spatial location of 2d lens spectra in science image¶

In [8]:
file_list_spec1d = []
frame_list = []
for i,p in enumerate(glob.glob(file_loc)):
    if 'spec1d' in p and '2M1310' in p:
        file_list_spec1d.append(p)
        f = p.split('_')
        frame = f[-4]+ '_' + f[-3][:4]
        frame_list.append(frame)

In [9]:
file_list_spec1d

['../galaxy_spectra/DEIMOS_data/2M1310_data/spec1d_d0613_0075-2M1310m1_DEIMOS_20210613T063837.766.fits',
 '../galaxy_spectra/DEIMOS_data/2M1310_data/spec1d_d0614_0075-2M1310m1_DEIMOS_2021Jun14T065013.891.fits',
 '../galaxy_spectra/DEIMOS_data/2M1310_data/spec1d_d0613_0074-2M1310m1_DEIMOS_20210613T060655.930.fits']

In [10]:
for fl, frm in zip(file_list_spec1d, frame_list):
    sp = pf.open(fl)
    #m = sp.slitinfo['objid']=='lens'
    nspec = sp[0].header['NSPEC']
    print(frm, nspec)
    #if frm =='d0613_0105':
        #pass
    #elif frm =='d0613_0106':
        #pass
    #else:
    for i in range(nspec):
        if 'lens' in sp[i+1].header['HIERARCH MASKDEF_OBJNAME']:
            print(sp[i+1].header['NAME'])

d0613_0075 125
SPAT1222-SLIT1232-DET02
SPAT1218-SLIT1227-DET06
d0614_0075 125
SPAT1222-SLIT1232-DET02
SPAT1218-SLIT1227-DET06
d0613_0074 121
SPAT1222-SLIT1232-DET02
SPAT1218-SLIT1227-DET06


## median image after sky subtraction

In [ ]:
13_74 2 - [1164, 1298] 6-[1154, 1298]; 13_75 2 - [1164, 1298] 6-[1154, 1298];
14_75 2 - [1164, 1298] 6-[1154, 1298]

In [37]:
xtrim_det = [[1164, 1298], [1154, 1298]]

spec_det_2 = []
spec_det_6 = []
for i, fl in enumerate(file_list):
    spec1 = spec2d.Spec2d(fl, hext=14, xtrim=xtrim_det[0])
    spec2 = spec2d.Spec2d(fl, hext=66, xtrim=xtrim_det[1])
    
    spec1.set_dispaxis('y')
    spec1.subtract_sky_2d(use_skymod=True)
    spec_det_2.append(spec1.data)
    
    spec2.set_dispaxis('y')
    spec2.subtract_sky_2d(use_skymod=True)
    spec_det_6.append(spec2.data)


Loading file ../galaxy_spectra/DEIMOS_data/2M1310_data/spec2d_d0613_0074-2M1310m1_DEIMOS_20210613T060655.930.fits
-----------------------------------------------
Read in 2-dimensional spectrum from ../galaxy_spectra/DEIMOS_data/2M1310_data/spec2d_d0613_0074-2M1310m1_DEIMOS_20210613T060655.930.fits (HDU=14)
The input dataset was trimmed
 xrange: 1164:1299.  yrange: 0:4096
Final data dimensions (x y): 135 x 4096

Dispersion axis:              x
N_pixels along dispersion axis: 135


Loading file ../galaxy_spectra/DEIMOS_data/2M1310_data/spec2d_d0613_0074-2M1310m1_DEIMOS_20210613T060655.930.fits
-----------------------------------------------
Read in 2-dimensional spectrum from ../galaxy_spectra/DEIMOS_data/2M1310_data/spec2d_d0613_0074-2M1310m1_DEIMOS_20210613T060655.930.fits (HDU=66)
The input dataset was trimmed
 xrange: 1154:1299.  yrange: 0:4096
Final data dimensions (x y): 145 x 4096

Dispersion axis:              x
N_pixels along dispersion axis: 145


Old value of dispaxis: x

Dis

In [38]:
median_im_det2 = np.median(np.asarray(spec_det_2), axis=0)
median_im_det6 = np.median(np.asarray(spec_det_6), axis=0)

In [39]:
pf.PrimaryHDU(median_im_det2).writeto('2M1310_DEIMOS_spectra/2M1310_median_image_det2.fits')
pf.PrimaryHDU(median_im_det6).writeto('2M1310_DEIMOS_spectra/2M1310_median_image_det6.fits')

### subtract sky and cosmic rays

In [49]:
extension = [14, 66]
trim = [[16, 119], [24, 119]]

median_im = ['2M1310_DEIMOS_spectra/2M1310_median_image_det2.fits', 
             '2M1310_DEIMOS_spectra/2M1310_median_image_det6.fits']

for fl, ofl1, ofl2 in zip(file_list, out_file_det2, out_file_det6):
        
    for ext, xtr, mdim in zip(extension, xtrim_det, median_im):
        print(ext, xtr, mdim)
        spec = spec2d.Spec2d(fl, hext=ext, xtrim=xtr)
        #spec2 = spec2d.Spec2d(fl, hext=66, xtrim=[970, 1080])
        spec.set_dispaxis('y')
        spec.subtract_sky_2d(use_skymod=True)
        spec.szap_new(infile=mdim)
        
        if ext==14:
            filled_data = spec.data[:, 16:120]
            mask = spec.szap_mask[:, 16:120]
            #print(filled_data.shape)
            ivar_im = pf.open(fl)[ext+1].data[spec.ymin:spec.ymax, spec.xmin:spec.xmax][:, 16:120]
            sky_im = pf.open(fl)[ext+2].data[spec.ymin:spec.ymax, spec.xmin:spec.xmax][:, 16:120]
            wav_im = pf.open(fl)[ext+7].data[spec.ymin:spec.ymax, spec.xmin:spec.xmax][:, 16:120]
            
        elif ext==66:
            filled_data = spec.data[:, 24:120]
            mask = spec.szap_mask[:, 24:120]
            #print(filled_data.shape)
            ivar_im = pf.open(fl)[ext+1].data[spec.ymin:spec.ymax, spec.xmin:spec.xmax][:, 24:120]
            sky_im = pf.open(fl)[ext+2].data[spec.ymin:spec.ymax, spec.xmin:spec.xmax][:, 24:120]
            wav_im = pf.open(fl)[ext+7].data[spec.ymin:spec.ymax, spec.xmin:spec.xmax][:, 24:120]
    
        ivar_im[mask] = 1.e-5
        #print(ivar_im.shape)
        
        prim_hdr = pf.open(fl)[0].header
        prim_hdr.append(('OBJECT', '2M1310-1714'))
        prim_hdr.append(('', 'Trimmed and cleaned data by Pritom'))
        phdu = pf.PrimaryHDU(header=prim_hdr)
        
        sci_hdr = pf.Header()
        sci_hdr.append(('OBJECT', '2M1310-1714'))
        sci_hdr.append(('IMTYPE', 'SCIENCE_IMAGE'))
        sci_im_hdu = pf.ImageHDU(data=filled_data, header=sci_hdr, name='SCI_IM')

        ivar_hdr = pf.Header()
        ivar_hdr.append(('IMTYPE', 'IVAR_IMAGE'))
        ivar_im_hdu = pf.ImageHDU(data=ivar_im, header=ivar_hdr, name='IVAR')

        sky_hdr = pf.Header()
        sky_hdr.append(('IMTYPE', 'SKY_MODEL'))
        sky_im_hdu = pf.ImageHDU(data=sky_im, header=sky_hdr, name='SKY_MOD')

        wav_hdr = pf.Header()
        wav_hdr.append(('IMTYPE', 'WAV_IMAGE'))
        wav_im_hdu = pf.ImageHDU(data=wav_im, header=wav_hdr, name='WAV_IM')

        mask_hdr = pf.Header()
        mask_hdr.append(('IMTYPE', 'MASK'))
        cmt = 'pixels masked as cosmic rays and filled with pixel values at same pixel'\
            'location from median image'
        mask_hdr.append(('', cmt))
        mask_hdu = pf.ImageHDU(data=mask, header=mask_hdr, name='MASK')

        hdul = pf.HDUList([phdu, sci_im_hdu, ivar_im_hdu, sky_im_hdu, wav_im_hdu, mask_hdu ])
        
        if ext==14:
            hdul.writeto(ofl1)
            print(ofl1)
        else:
            hdul.writeto(ofl2)
            print(ofl2)

14 [1164, 1298] 2M1310_DEIMOS_spectra/2M1310_median_image_det2.fits

Loading file ../galaxy_spectra/DEIMOS_data/2M1310_data/spec2d_d0613_0074-2M1310m1_DEIMOS_20210613T060655.930.fits
-----------------------------------------------
Read in 2-dimensional spectrum from ../galaxy_spectra/DEIMOS_data/2M1310_data/spec2d_d0613_0074-2M1310m1_DEIMOS_20210613T060655.930.fits (HDU=14)
The input dataset was trimmed
 xrange: 1164:1299.  yrange: 0:4096
Final data dimensions (x y): 135 x 4096

Dispersion axis:              x
N_pixels along dispersion axis: 135


Old value of dispaxis: x

Dispersion axis:              y
N_pixels along dispersion axis: 4096



pypeit generated sky model will be used for sky subtraction


From this point sky subtracted data will be used
 Spectrum Start:     0.00
 Spectrum End:     4095.00
 Dispersion (1st pixel):   1.00
 Dispersion (average):      1.00


From now cosmic ray rejected data will be used
2M1310_DEIMOS_spectra/2M1310_deimos_det2_d0613_0074_clean.fits
66 [115

In [47]:
spec.data.shape

(4096, 145)